# EDA

Here we are gonna dig inside our data and see if we find missing values, wrong data types etc...

# Table of Contents
1. [Step 1: the shape of data](#Step-1:-the-shape-of-data)
2. [Step 2: Analyzing datatypes](#Step-2:-Analyzing-datatypes)
3. [Step 3 : Check for duplicated entries](#Step-3-:-Check-for-duplicated-entries)
4. [Step 4: Null values](#Step-4:-Null-values)

## Step 1: the shape of data

In [1]:
import pandas as pd
df = pd.read_csv('kc_house_data.csv')
df.head()

,id,date,price,bedrooms,bathrooms,sqft_living,sqft_lot,floors,waterfront,view,...,grade,sqft_above,sqft_basement,yr_built,yr_renovated,zipcode,lat,long,sqft_living15,sqft_lot15
0,7129300520,10/13/2014,221900.0,3,1.00,1180,5650,1.0,NaN,0.0,...,7,1180,0.0,1955,0.0,98178,47.5112,-122.257,1340,5650
1,6414100192,12/9/2014,538000.0,3,2.25,2570,7242,2.0,0.0,0.0,...,7,2170,400.0,1951,1991.0,98125,47.7210,-122.319,1690,7639
2,5631500400,2/25/2015,180000.0,2,1.00,770,10000,1.0,0.0,0.0,...,6,770,0.0,1933,NaN,98028,47.7379,-122.233,2720,8062
3,2487200875,12/9/2014,604000.0,4,3.00,1960,5000,1.0,0.0,0.0,...,7,1050,910.0,1965,0.0,98136,47.5208,-122.393,1360,5000
4,1954400510,2/18/2015,510000.0,3,2.00,1680,8080,1.0,0.0,0.0,...,8,1680,0.0,1987,0.0,98074,47.6168,-122.045,1800,7503


In [2]:
df.shape

(21597, 21)

### Conclusion
All looks good

## Step 2: Analyzing datatypes

**Check Column naming**

In [3]:
df.columns

Index(['id', 'date', 'price', 'bedrooms', 'bathrooms', 'sqft_living',
       'sqft_lot', 'floors', 'waterfront', 'view', 'condition', 'grade',
       'sqft_above', 'sqft_basement', 'yr_built', 'yr_renovated', 'zipcode',
       'lat', 'long', 'sqft_living15', 'sqft_lot15'],
      dtype='object')

__Notes__ Column name check - need to some of the columns have different names in column readme file, 
bedrooms vs bedroomsNumber, bathrooms vs bathroomsNumber, etc...

**Check data types**

In [4]:
df.dtypes

id                 int64
date              object
price            float64
bedrooms           int64
bathrooms        float64
sqft_living        int64
sqft_lot           int64
floors           float64
waterfront       float64
view             float64
condition          int64
grade              int64
sqft_above         int64
sqft_basement     object
yr_built           int64
yr_renovated     float64
zipcode            int64
lat              float64
long             float64
sqft_living15      int64
sqft_lot15         int64
dtype: object

In [5]:
# date column needs a to_datetime (is currently string)
# bathrooms 2.25 doesn't make sense, 2.5 does - according to lindsey, 1/4  and 3/4 baths are a thing 
# floors should be int not float
# waterfront is a float, should be bool
# what do the ints from 3 to 5 mean in terms of condition? 3=good? 5-bad? or vice versa? condition affects price?
# same for grade, get more info, info affects price?
# sqfoot above mean above ground (not including basement) 
# sqfoot basement may have impact on price, but not as much as above ground sq footage (is float, needs to be int).
# We can see that it is a object where it should be an integer like the other surface measures.
# yrbuilt - datetime? or in int ok?
# yrrenovated shouldn't be a float
# sqft living & lot based on 15 nearest neighbors

In [6]:
# sqft_basement has a '?' string as its Null value
sqft_basement = df['sqft_basement']
pd.to_numeric(sqft_basement)

ValueError: Unable to parse string "?" at position 6

In [7]:
# How many empty sqft_basement measures do we have?
len(df.loc[df['sqft_basement'] == '?'])

454

In [8]:
# Defines most commonly found value for sqft_basement
sqft_basement.value_counts().head()

0.0      12826
?          454
600.0      217
500.0      209
700.0      208
Name: sqft_basement, dtype: int64

**Conclusion** : Let's assume that those with unknown sqft_basement are actually with no basement and set to 0

In [ ]:
# Yr renovated
yr_renovated = df['yr_renovated']
yr_renovated = pd.to_numeric(yr_renovated)

In [ ]:
yr_renovated.isna().sum()

In [ ]:
yr_renovated.isnull().sum()

In [ ]:
# Defines most commonly found value for yr_renovated
yr_renovated.value_counts().head()

**Conclusion** As there are lot of values that can't be ignored in yr_renovated, let's set the NaN values to the yr_built or assumes that they have never been renovated. We will perfom the datatype change to datetime after.

## Step 3 : Check for duplicated entries

In [9]:
# Checking for duplicated entries in IDs.
df.loc[df.duplicated(['id'], keep=False)].sort_values(by=['id'], ascending=True)

,id,date,price,bedrooms,bathrooms,sqft_living,sqft_lot,floors,waterfront,view,...,grade,sqft_above,sqft_basement,yr_built,yr_renovated,zipcode,lat,long,sqft_living15,sqft_lot15
2495,1000102,4/22/2015,300000.0,6,3.00,2400,9373,2.0,0.0,0.0,...,7,2400,0.0,1991,0.0,98002,47.3262,-122.214,2060,7316
2494,1000102,9/16/2014,280000.0,6,3.00,2400,9373,2.0,NaN,0.0,...,7,2400,0.0,1991,0.0,98002,47.3262,-122.214,2060,7316
16800,7200179,10/16/2014,150000.0,2,1.00,840,12750,1.0,0.0,0.0,...,6,840,0.0,1925,0.0,98055,47.4840,-122.211,1480,6969
16801,7200179,4/24/2015,175000.0,2,1.00,840,12750,1.0,0.0,0.0,...,6,840,0.0,1925,NaN,98055,47.4840,-122.211,1480,6969
11422,109200390,10/20/2014,250000.0,3,1.75,1480,3900,1.0,0.0,0.0,...,7,1480,0.0,1980,0.0,98023,47.2977,-122.367,1830,6956
11421,109200390,8/20/2014,245000.0,3,1.75,1480,3900,1.0,0.0,0.0,...,7,1480,0.0,1980,0.0,98023,47.2977,-122.367,1830,6956
12406,123039336,12/8/2014,244900.0,1,1.00,620,8261,1.0,0.0,0.0,...,5,620,0.0,1939,NaN,98106,47.5138,-122.364,1180,8244
12405,123039336,6/11/2014,148000.0,1,1.00,620,8261,1.0,0.0,0.0,...,5,620,0.0,1939,0.0,98106,47.5138,-122.364,1180,8244
7786,251300110,1/14/2015,358000.0,3,2.25,2510,12013,2.0,0.0,0.0,...,8,2510,0.0,1988,0.0,98003,47.3473,-122.314,1870,8017
7785,251300110,7/31/2014,225000.0,3,2.25,2510,12013,2.0,0.0,0.0,...,8,2510,0.0,1988,0.0,98003,47.3473,-122.314,1870,8017


**Conclusion** : It seems like the properties listed have been sold multiple times during the period.
It might be helpful to keep the information and not drop the duplicated values has they happen in different times.

## Step 4: Null values

In [10]:
df.describe()

,id,price,bedrooms,bathrooms,sqft_living,sqft_lot,floors,waterfront,view,condition,grade,sqft_above,yr_built,yr_renovated,zipcode,lat,long,sqft_living15,sqft_lot15
count,2.159700e+04,2.159700e+04,21597.000000,21597.000000,21597.000000,2.159700e+04,21597.000000,19221.000000,21534.000000,21597.000000,21597.000000,21597.000000,21597.000000,17755.000000,21597.000000,21597.000000,21597.000000,21597.000000,21597.000000
mean,4.580474e+09,5.402966e+05,3.373200,2.115826,2080.321850,1.509941e+04,1.494096,0.007596,0.233863,3.409825,7.657915,1788.596842,1970.999676,83.636778,98077.951845,47.560093,-122.213982,1986.620318,12758.283512
std,2.876736e+09,3.673681e+05,0.926299,0.768984,918.106125,4.141264e+04,0.539683,0.086825,0.765686,0.650546,1.173200,827.759761,29.375234,399.946414,53.513072,0.138552,0.140724,685.230472,27274.441950
min,1.000102e+06,7.800000e+04,1.000000,0.500000,370.000000,5.200000e+02,1.000000,0.000000,0.000000,1.000000,3.000000,370.000000,1900.000000,0.000000,98001.000000,47.155900,-122.519000,399.000000,651.000000
25%,2.123049e+09,3.220000e+05,3.000000,1.750000,1430.000000,5.040000e+03,1.000000,0.000000,0.000000,3.000000,7.000000,1190.000000,1951.000000,0.000000,98033.000000,47.471100,-122.328000,1490.000000,5100.000000
50%,3.904930e+09,4.500000e+05,3.000000,2.250000,1910.000000,7.618000e+03,1.500000,0.000000,0.000000,3.000000,7.000000,1560.000000,1975.000000,0.000000,98065.000000,47.571800,-122.231000,1840.000000,7620.000000
75%,7.308900e+09,6.450000e+05,4.000000,2.500000,2550.000000,1.068500e+04,2.000000,0.000000,0.000000,4.000000,8.000000,2210.000000,1997.000000,0.000000,98118.000000,47.678000,-122.125000,2360.000000,10083.000000
max,9.900000e+09,7.700000e+06,33.000000,8.000000,13540.000000,1.651359e+06,3.500000,1.000000,4.000000,5.000000,13.000000,9410.000000,2015.000000,2015.000000,98199.000000,47.777600,-121.315000,6210.000000,871200.000000


In [ ]:
# count for boolean values, can give us a better idea of distribution
# max of floors is 3.5, why .5 floors? is that the attic?
# std normalize scale factors? compare?

In [11]:
df.isna().sum()

id                  0
date                0
price               0
bedrooms            0
bathrooms           0
sqft_living         0
sqft_lot            0
floors              0
waterfront       2376
view               63
condition           0
grade               0
sqft_above          0
sqft_basement       0
yr_built            0
yr_renovated     3842
zipcode             0
lat                 0
long                0
sqft_living15       0
sqft_lot15          0
dtype: int64

**Check waterfront and view missing values**

In [ ]:
# Check different values for waterfront
waterfront_ds = df['waterfront']
print(waterfront_ds.value_counts())
print('Waterfront - Total of missing values : ', waterfront_ds.isna().sum())

In [ ]:
# Check different values for view
view_ds = df['view']
print(view_ds.value_counts())
print('view - Total of missing values : ', view_ds.isna().sum())

**Conclusion** As described earlier, we already identified missing data under yr_renovated and sqft_basement when checking datatype. Now we see also that the dataset contains missing data for waterfront and view. As the number of NaN  for "waterfront" is huge we gonna keep the entries and replace the NaN by the most common value 0. The same strategy is used for "view" variable

In [ ]:
# Location is clearly important in real-estate, so we'd like to look at zip code and latitude/longitude, 
# but to avoid multicollinearity we don't want to select more than one variable.
# We will make separate models for these two measures of location and see which is a better predictor of value.
# The third question we'd like to pose is how well the square footage metrics predict the price of homes.